In [53]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [54]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os. getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [55]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [56]:
def chat(message, history):
    messages = [{"role":"system", "content" : system_message}]
    for human, assistant in history:
        messages.append ({"role": "user", "content": human})
        messages.append ({"role": "assistant", "content": assistant})
    messages.append({"role":"user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages = messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat).launch()

C:\Users\QIRAT\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [57]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "sialkot": "$399"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    
    return ticket_prices.get(city, "Unknown")

In [58]:
get_ticket_price("Sialkot")

Tool get_ticket_price called for Sialkot


'$399'

In [59]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [60]:
tools = [{"type":"function", "function":price_function}]

In [61]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}]
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    messages.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [62]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": message.tool_calls[0].id
    }
    return response, city

In [63]:
gr.ChatInterface(fn=chat).launch()

C:\Users\QIRAT\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


# DALL-E-3, the image generation model behind GPT-4o

In [76]:
import base64
from io import BytesIO
from PIL import Image


In [86]:
def ImageGeneration(info):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {info}, showing tourist spots and everything unique about {info}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [83]:
image = ImageGeneration("Burj Khalifa")
display(image)

NameError: name 'city' is not defined

## Audio


In [64]:
import pygame
import io
from time import sleep

def talker(message):
    try:
        # Initialize pygame mixer
        pygame.mixer.init()
        
        # Create speech
        print("Generating speech...")
        response = openai.audio.speech.create(
            model="tts-1",
            voice="nova",
            input=message
        )
        
        # Load audio data into memory
        print("Processing audio...")
        audio_data = io.BytesIO(response.content)
        
        # Load and play the audio
        print("Playing audio...")
        pygame.mixer.music.load(audio_data)
        pygame.mixer.music.play()
        
        # Wait for the audio to finish playing
        while pygame.mixer.music.get_busy():
            sleep(0.1)
            
        print("Audio playback completed!")
        
        # Cleanup
        pygame.mixer.quit()
        
    except Exception as e:
        print(f"Error: {type(e).__name__} - {str(e)}")
        pygame.mixer.quit()
        return None

In [65]:
#print(f"Temporary directory1: {tempfile.tempdir}")
talker("Well, hi there")

Generating speech...
Processing audio...
Playing audio...
Audio playback completed!


# Agent Framework

In [80]:
def chat(message, history):
    image = None
    conversation = [{"role": "system", "content":system_message}]
    for human, assistant in history:
        conversation.append({"role": "user", "content": human})
        conversation.append({"role": "assistant", "content": assistant})
    conversation.append({"role": "user", "content": message})
    response = openai.chat.completions.create(model=MODEL, messages=conversation, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = tool_call = response.choices[0].message
        response,city = handle_tool_call(message)
        conversation.append(message)
        conversation.append(response)
        image = ImageGeneration(city)
        response = openai.chat.completions.create(model = MODEL, messages = conversation)

    reply = response.choices[0].message.content
    talker(reply)
    return reply, image
        
    

In [87]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500)
        image_output = gr.Image(height=500)
    with gr.Row():
        msg = gr.Textbox(label="Chat with our AI Assistant:")
    with gr.Row():
        clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        user_message = history[-1][0]
        bot_message, image = chat(user_message, history[:-1])
        history[-1][1] = bot_message
        return history, image

    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, [chatbot, image_output]
    )
    clear.click(lambda: None, None, chatbot, queue=False)

ui.launch()

C:\Users\QIRAT\anaconda3\envs\llms\Lib\site-packages\gradio\components\chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


Generating speech...
Processing audio...
Playing audio...
Audio playback completed!
Tool get_ticket_price called for Sialkot
Generating speech...
Processing audio...
Playing audio...
Audio playback completed!
